In [ ]:
# import all important libraries

import pandas as pd
import requests
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#keys
API_KEY = 'xxxxx'
CHANNEL_ID = "xxxxx"

In [ ]:
# function to get video statistics
 
def get_video_details(video_id):

  url_video_stats = 'https://www.googleapis.com/youtube/v3/videos?id='+video_id+'&part=statistics&key='+API_KEY
  response_video_stats = requests.get(url_video_stats).json()

  view_count = response_video_stats['items'][0]['statistics']['viewCount']
  like_count = response_video_stats['items'][0]['statistics']['likeCount']
  comment_count = response_video_stats['items'][0]['statistics']['commentCount']

  return view_count,like_count,comment_count

In [ ]:
# function to get video details from API call

def get_videos(df):
    #make API call
    pageToken = ''
    while 1:
        url = 'https://www.googleapis.com/youtube/v3/search?key='+API_KEY+'&channel_id='+CHANNEL_ID+'&part=snippet,id&order=date&maxResults=10000&'+pageToken
        response = requests.get(url).json()
  
        time.sleep(1) #give it a second before starting the for loop

  
        
        for video in response['items']:
            if video['id']['kind'] == 'youtube#video':
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace('&amp;','')
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split('T')[0]
                
                view_count,like_count,comment_count = get_video_details(video_id)
      

      # save data to df
                df = df.append({'video_id':video_id,'video_title':video_title,
                       'upload_date':upload_date,'view_count':view_count,
                       'like_count':like_count,'comment_count':comment_count},ignore_index=True)
    
        try:
            if response['nextPageToken'] != None: #if none, it means it reached the last page and break out of it
                pageToken = "pageToken=" + response['nextPageToken']

        except:
            break
    
    return df
 

In [ ]:
# create dataframe to store all video details
df = pd.DataFrame(columns=['video_id','video_title','upload_date','view_count','like_count','comment_count'])


# make a call
df = get_videos(df)

In [ ]:
#if you want you can save it as csv file

df.to_csv('channel', encoding='utf-8', index=False,)